<a href="https://colab.research.google.com/github/MunYeong0723/NLP_mini-project/blob/master/Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install scikit-surprise

In [2]:
import pandas as pd
from ast import literal_eval
from surprise import *
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import glob

In [3]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 300)

In [4]:
movie_path_list = glob.glob('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/watcha_csv_data/watcha_movie_data/*.csv')

movie_list = pd.DataFrame(columns=['movie_code', 'title', 'year', 'genres', 'ratings_avg'])
for movie_path in movie_path_list :
    tmp = pd.read_csv(movie_path, encoding='CP949')
    tmp = tmp[['movie_code', 'title', 'year', 'genres', 'ratings_avg']]
    movie_list = pd.concat([movie_list, tmp], axis=0)

movie_list

,movie_code,title,year,genres,ratings_avg
0,md6B3Ad,대부 일대기,1992,"['드라마', '범죄']",4.4
1,mNdKBkO,쇼생크 탈출,1994,['드라마'],4.4
2,mV539wO,인생은 아름다워,1997,"['드라마', '코미디']",4.4
3,mW4LYBo,하이큐!! 콘셉트의 싸움,2017,"['애니메이션', '스포츠']",4.4
4,m5r0pVO,12인의 노한 사람들,1997,"['드라마', '범죄']",4.2
...,...,...,...,...,...
94,mW9lkG5,댄저러스,1935,['드라마'],3.3
95,mXOg7gO,어느날 밤에 생긴 일,1934,['로맨틱코미디'],3.7
96,mdBXkPW,아침의 영광,1933,['드라마'],2.7
97,md6my0O,마델론의 비극,1931,['드라마'],0.0


In [5]:
user_movie_rating_origin = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/watcha_csv_data/watcha_user_data/watcha_top_movie_user_data.csv')
user_movie_rating_origin = user_movie_rating_origin[['user_code', 'movie_code', 'user_rating']]
user_movie_rating_origin.columns = ['user_code', 'movie_code', 'movie_rating']

user_movie_rating_origin

,user_code,movie_code,movie_rating
0,WRQxDkw965dl9,md6B3Ad,4.5
1,nkPvrb8wrvgar,md6B3Ad,4.5
2,WwRqoR4OWxlzB,md6B3Ad,5.0
3,YaR5Y0ZWKvgX1,md6B3Ad,5.0
4,17ovVbgA9vzyn,md6B3Ad,5.0
...,...,...,...
4360,WRQxDkw965dl9,m5ZXrxO,3.0
4361,2mwvgN07PEqMa,m5ZXrxO,4.5
4362,Qgy51rmX7xDjk,m5ZXrxO,3.5
4363,WRQxDmg4Gqdl9,m5ZXrxO,4.0


In [6]:
user_movie_wish_origin = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/watcha_csv_data/watcha_user_data/watcha_top_movie_user_wish_data.csv')
user_movie_wish_origin = user_movie_wish_origin[['user_code', 'movie_code']]
user_movie_wish_origin

,user_code,movie_code
0,WRQxDkw965dl9,mOPNrY5
1,WRQxDkw965dl9,mOP6lD5
2,WRQxDkw965dl9,mXOgb0W
3,WRQxDkw965dl9,m9dBxqW
4,WRQxDkw965dl9,mOPoG8D
...,...,...
47778,6ewxaBeAGqQ1m,mWLmy6O
47779,6ewxaBeAGqQ1m,m5napDd
47780,6ewxaBeAGqQ1m,m5XPw15
47781,6ewxaBeAGqQ1m,mOAaylO


In [ ]:
# 정수 인코딩
movie_code = movie_list['movie_code'].unique().tolist()
movie_to_index = {x : i for i,x in enumerate(movie_code)}
index_to_movie = {i : x for i,x in enumerate(movie_code)}

user_code = pd.merge(user_movie_rating_origin['user_code'], user_movie_wish_origin['user_code'], on='user_code', how='outer')
user_code = user_code['user_code'].unique().tolist()
user_to_index = {x : i for i,x in enumerate(user_code)}
index_to_user = {i : x for i,x in enumerate(user_code)}

print(movie_to_index)
print(index_to_movie)
print()
print(user_to_index)
print(index_to_user)

In [8]:
# 정수 인코딩한 것으로 code 바꾸기
movie_list['movie_code'] = movie_list['movie_code'].map(movie_to_index)

user_movie_rating = user_movie_rating_origin.copy()
user_movie_wish = user_movie_wish_origin.copy()

user_movie_rating['user_code'] = user_movie_rating['user_code'].map(user_to_index)
user_movie_rating['movie_code'] = user_movie_rating['movie_code'].map(movie_to_index)

user_movie_wish['user_code'] = user_movie_wish['user_code'].map(user_to_index)
user_movie_wish['movie_code'] = user_movie_wish['movie_code'].map(movie_to_index)

In [9]:
display(movie_list)
display(user_movie_rating)
display(user_movie_wish)

,movie_code,title,year,genres,ratings_avg
0,0,대부 일대기,1992,"['드라마', '범죄']",4.4
1,1,쇼생크 탈출,1994,['드라마'],4.4
2,2,인생은 아름다워,1997,"['드라마', '코미디']",4.4
3,3,하이큐!! 콘셉트의 싸움,2017,"['애니메이션', '스포츠']",4.4
4,4,12인의 노한 사람들,1997,"['드라마', '범죄']",4.2
...,...,...,...,...,...
94,1060,댄저러스,1935,['드라마'],3.3
95,430,어느날 밤에 생긴 일,1934,['로맨틱코미디'],3.7
96,1061,아침의 영광,1933,['드라마'],2.7
97,1062,마델론의 비극,1931,['드라마'],0.0


,user_code,movie_code,movie_rating
0,0,0,4.5
1,1,0,4.5
2,2,0,5.0
3,3,0,5.0
4,4,0,5.0
...,...,...,...
4360,0,299,3.0
4361,1271,299,4.5
4362,212,299,3.5
4363,660,299,4.0


,user_code,movie_code
0,0,NaN
1,0,NaN
2,0,NaN
3,0,1016.0
4,0,NaN
...,...,...
47778,282,NaN
47779,282,NaN
47780,282,NaN
47781,282,528.0


In [10]:
# 중복 제거
movie_list = movie_list.drop_duplicates(['movie_code']).reset_index(drop=True)

# 결측치 제거 -> movie_list에 없는 movie일 때
# list에 있는 영화만을 가지고 train하기 위해 일단 list에 없는 영화에 대한 (평가 & wish) 는 제거
user_movie_rating.dropna(axis=0, inplace=True)
user_movie_wish.dropna(axis=0, inplace=True)

# reset_index
user_movie_rating = user_movie_rating.reset_index(drop=True)
user_movie_wish = user_movie_wish.reset_index(drop=True)

In [11]:
display(movie_list)
display(user_movie_rating)
display(user_movie_wish)

,movie_code,title,year,genres,ratings_avg
0,0,대부 일대기,1992,"['드라마', '범죄']",4.4
1,1,쇼생크 탈출,1994,['드라마'],4.4
2,2,인생은 아름다워,1997,"['드라마', '코미디']",4.4
3,3,하이큐!! 콘셉트의 싸움,2017,"['애니메이션', '스포츠']",4.4
4,4,12인의 노한 사람들,1997,"['드라마', '범죄']",4.2
...,...,...,...,...,...
1059,1059,제저벨,1938,['드라마'],3.3
1060,1060,댄저러스,1935,['드라마'],3.3
1061,1061,아침의 영광,1933,['드라마'],2.7
1062,1062,마델론의 비극,1931,['드라마'],0.0


,user_code,movie_code,movie_rating
0,0,0,4.5
1,1,0,4.5
2,2,0,5.0
3,3,0,5.0
4,4,0,5.0
...,...,...,...
4316,0,299,3.0
4317,1271,299,4.5
4318,212,299,3.5
4319,660,299,4.0


,user_code,movie_code
0,0,1016.0
1,0,33.0
2,0,458.0
3,0,350.0
4,0,517.0
...,...,...
6938,282,819.0
6939,282,932.0
6940,282,348.0
6941,282,405.0


In [12]:
# user 한 명 추가
repre_rating = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/rating_data_pre.csv')
repre_wish = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/wish_data_pre.csv')

# title -> movie_code
title_to_code = movie_list[['title', 'movie_code']].set_index("title").T.to_dict('index')
title_to_code = title_to_code['movie_code']

# data preprocessing
repre_rating['movie_code'] = repre_rating['제목'].map(title_to_code)
repre_rating = repre_rating.dropna(axis=0)
repre_rating.columns = ['title', 'genres', 'total_avg', 'movie_rating', 'movie_code']

repre_wish['movie_code'] = repre_wish['제목'].map(title_to_code)
repre_wish = repre_wish.dropna(axis=0)
repre_wish.columns = ['title', 'genres', 'total_avg', 'movie_code']

repre_rating['user_code'] = [1653 for _ in range(len(repre_rating))]
repre_wish['user_code'] = [1653 for _ in range(len(repre_wish))]

repre_rating = repre_rating[['user_code', 'movie_code', 'movie_rating']]
repre_wish = repre_wish[['user_code', 'movie_code']]

# 중복 제거
repre_rating = repre_rating.drop_duplicates(['movie_code'])
repre_wish = repre_wish.drop_duplicates(['movie_code'])

# user_movie_rating 에 합치기
user_movie_rating = user_movie_rating.append(repre_rating)
user_movie_wish = user_movie_wish.append(repre_wish)

# user_to_code & code_to_user 에 추가하기
user_to_index['represent'] = 1653
index_to_user[1653] = 'represent'

display(user_movie_rating)
display(user_movie_wish)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


,user_code,movie_code,movie_rating
0,0,0.0,4.5
1,1,0.0,4.5
2,2,0.0,5.0
3,3,0.0,5.0
4,4,0.0,5.0
...,...,...,...
5044,1653,227.0,3.5
5055,1653,383.0,3.5
5056,1653,282.0,3.5
5088,1653,304.0,3.5


,user_code,movie_code
0,0,1016.0
1,0,33.0
2,0,458.0
3,0,350.0
4,0,517.0
...,...,...
4936,1653,858.0
5251,1653,952.0
5259,1653,274.0
5262,1653,700.0


In [13]:
# movie_code를 int로 바꾸기
user_movie_rating['movie_code'] = user_movie_rating['movie_code'].apply(int)
user_movie_wish['movie_code'] = user_movie_wish['movie_code'].apply(int)

display(user_movie_rating)
display(user_movie_wish)

,user_code,movie_code,movie_rating
0,0,0,4.5
1,1,0,4.5
2,2,0,5.0
3,3,0,5.0
4,4,0,5.0
...,...,...,...
5044,1653,227,3.5
5055,1653,383,3.5
5056,1653,282,3.5
5088,1653,304,3.5


,user_code,movie_code
0,0,1016
1,0,33
2,0,458
3,0,350
4,0,517
...,...,...
4936,1653,858
5251,1653,952
5259,1653,274
5262,1653,700


In [14]:
# shuffle
user_movie_rating = user_movie_rating.sample(frac=1).reset_index(drop=True)
display(user_movie_rating)

reader = Reader(rating_scale=(0.0, 5.0))
data = Dataset.load_from_df(df=user_movie_rating[['user_code', 'movie_code', 'movie_rating']], reader=reader)
display(data)

,user_code,movie_code,movie_rating
0,368,35,5.0
1,279,269,4.0
2,1118,173,3.5
3,956,142,5.0
4,1653,42,4.0
...,...,...,...
4732,327,166,3.5
4733,145,204,4.0
4734,1653,454,3.5
4735,41,6,5.0


## fit & predict

In [15]:
# 여러 알고리즘을 비교하여 RMSE가 가장 적은 알고리즘을 선택
# SVDpp를 선택하였을 때의 loss가 제일 적지만 시간이 오래 걸림 => 그 다음으로 loss가 적은 SVD를 선택
algo_str = "SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering"
algo = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(),KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

benchmark = []
# iterate all algorithms
for i in range(len(algo)) :
    results = cross_validate(algo[i], data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
    # print(results)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([algo_str.split(", ")[i]], index=["Algorithm"]))
    benchmark.append(tmp)

pd.DataFrame(benchmark).set_index("Algorithm").sort_values(["test_rmse", "test_mae"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.639659,0.473448,2.109778,0.030426
SVD,0.647067,0.485104,0.250604,0.005734
BaselineOnly,0.647170,0.489618,0.007716,0.004259
KNNBaseline,0.665030,0.492102,0.066334,0.019649
KNNWithMeans,0.714298,0.527468,0.073809,0.019186
KNNWithZScore,0.721366,0.531925,0.108620,0.020796
CoClustering,0.738634,0.558740,0.220155,0.004472
KNNBasic,0.744535,0.579546,0.056458,0.018661
SlopeOne,0.759221,0.566765,0.025807,0.017793


In [16]:
param_grid = {
    'n_epochs':[5,10,50,100],
    'n_factors' : [10,20,50,100]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.633716284238031
{'n_epochs': 100, 'n_factors': 50}


In [17]:
user_id = 1653
print(user_movie_rating['user_code'].unique())

[ 368  279 1118 ... 1560  818 1351]


In [18]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [19]:
preds = pd.DataFrame(columns=['pred_est', 'movie_code'])
for i in movie_list['movie_code'].unique() :
    pred = algo.predict(str(user_id), i)
    # preds.append((pred.est, pred.iid))

    preds = preds.append({
        'pred_est' : pred.est,
        'movie_code' : pred.iid
    }, ignore_index=True)

preds['movie_code'] = preds['movie_code'].apply(int)
preds = preds.sort_values(by='pred_est', ascending=False)
preds

,pred_est,movie_code
16,4.922331,16
11,4.819776,11
172,4.770180,172
52,4.769217,52
22,4.754807,22
...,...,...
274,3.552872,274
451,3.542086,451
916,3.392738,916
18,3.337505,18


## 높게 평가할 영화 + 보고 싶은 영화

In [21]:
user_wish = user_movie_wish[user_movie_wish['user_code'] == user_id]

tmp_df = pd.merge(user_wish, preds, on='movie_code', how='right')
tmp_df = tmp_df.drop_duplicates(['movie_code'])

top_10 = tmp_df.iloc[:10]
top_10 = top_10.sort_values(by='pred_est', ascending=False)
top_10

,user_code,movie_code,pred_est
2,1653.0,243,4.465877
5,1653.0,101,4.309836
3,1653.0,272,4.241756
6,1653.0,922,4.215036
1,1653.0,1056,4.214587
7,1653.0,351,4.214587
8,1653.0,593,4.214587
9,1653.0,435,4.186905
0,1653.0,215,4.163145
4,1653.0,103,4.133765


In [22]:
pd.merge(top_10['movie_code'], movie_list, on='movie_code', how='left')

,movie_code,title,year,genres,ratings_avg
0,243,더 폴: 오디어스와 환상의 문,2006,"['모험', '드라마', '판타지']",4.0
1,101,루스 베이더 긴즈버그: 나는 반대한다,2018,['다큐멘터리'],4.2
2,272,1917,2019,"['드라마', '전쟁']",4.2
3,922,드라이빙 미스 데이지,1989,"['코미디', '드라마']",3.6
4,1056,가스등,1944,"['범죄', '드라마', '미스터리', '스릴러']",3.8
5,351,필로우 토크,1959,"['로맨스', '코미디']",3.5
6,593,허슬러,1961,"['스포츠', '드라마']",3.6
7,435,조커,2019,"['스릴러', '드라마', '범죄']",3.9
8,215,디지몬 어드벤처 1기: 운명적 만남,1999,"['애니메이션', '가족', '액션', '모험', '판타지', 'SF', '스릴러']",4.1
9,103,포드 V 페라리,2019,"['액션', '드라마']",4.0


## 실제 rating과 비교해보기

In [23]:
user_1653 = user_movie_rating[user_movie_rating['user_code'] == user_id]

real_pred = pd.merge(user_1653, preds, on='movie_code')
real_pred = real_pred[['movie_code', 'movie_rating', 'pred_est']]
real_pred.columns = ['movie_code', 'real_est', 'pred_est']
real_pred.sort_values(by='pred_est', ascending=False)

,movie_code,real_est,pred_est
246,11,4.0,4.819776
383,52,4.0,4.769217
223,22,4.0,4.754807
163,2,4.0,4.754357
139,159,4.0,4.744019
...,...,...,...
112,274,3.5,3.552872
361,451,1.5,3.542086
237,916,1.0,3.392738
252,18,4.0,3.337505


## user_name 입력받아서 추천 list 출력하기

In [24]:
user_name_list = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/watcha_csv_data/watcha_user_data/watcha_top_movie_user_data.csv')
user_name_list = user_name_list[['user_code', 'user_name']]
user_name_list = user_name_list.append({
                                            'user_code' : 'represent',
                                            'user_name' : '평론가님'
                                        }, ignore_index=True)

# name_to_code dictionary
name_to_code = {}
for i in range(len(user_name_list)):
    user = user_name_list.iloc[i]
    name_to_code[user['user_name']] = user['user_code']

user_name_list.head()

,user_code,user_name
0,WRQxDkw965dl9,이호중
1,nkPvrb8wrvgar,Jay Oh
2,WwRqoR4OWxlzB,나쁜피
3,YaR5Y0ZWKvgX1,김하빈
4,17ovVbgA9vzyn,kimikimi


In [25]:
def predict_top10(input_name):
    user_code = name_to_code[input_name]
    user_id = user_to_index[user_code]

    # predict
    preds = pd.DataFrame(columns=['pred_est', 'movie_code'])
    for i in movie_list['movie_code'].unique() :
        pred = algo.predict(str(user_id), i)
        # preds.append((pred.est, pred.iid))

        preds = preds.append({
            'pred_est' : pred.est,
            'movie_code' : pred.iid
        }, ignore_index=True)

    preds['movie_code'] = preds['movie_code'].apply(int)
    preds = preds.sort_values(by='pred_est', ascending=False)

    # top 10 + wishList
    user_wish = user_movie_wish[user_movie_wish['user_code'] == user_id]

    tmp_df = pd.merge(user_wish, preds, on='movie_code', how='right')
    tmp_df = tmp_df.drop_duplicates(['movie_code'])
    
    top_10 = tmp_df.iloc[:10]
    top_10 = top_10.sort_values(by='pred_est', ascending=False)

    return pd.merge(top_10['movie_code'], movie_list, on='movie_code', how='left')

In [28]:
input_name = input()
predict_top10(input_name)

이호중


,movie_code,title,year,genres,ratings_avg
0,33,원더,2017,"['가족', '드라마']",4.2
1,458,애정만세,1994,['드라마'],3.8
2,132,쓰리 빌보드,2017,"['범죄', '드라마']",4.0
3,1016,몬스터,2003,"['범죄', '드라마', '스릴러']",3.6
4,231,인생,1994,['드라마'],4.0
5,350,아파트 열쇠를 빌려드립니다,1960,"['로맨틱코미디', '드라마', '코미디']",3.9
6,564,에비에이터,2004,"['드라마', '모험', '로맨스']",3.5
7,379,셰이프 오브 워터: 사랑의 모양,2017,"['모험', '스릴러', '판타지', '로맨스', '드라마']",3.9
8,517,바톤 핑크,1991,"['코미디', '드라마']",3.7
9,120,히든 피겨스,2016,"['역사', '드라마']",4.0
